In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df_tcpstream = pd.read_csv('data/bench_tcpstream.csv', delimiter=';', )
df_file = pd.read_csv('data/bench_file.csv', delimiter=';')

In [ ]:
tcpstream_means = df_tcpstream.mean()
file_means = df_file.mean()

In [ ]:
tcpstream_means['io_time'] = tcpstream_means['read_io_time'] + tcpstream_means['write_io_time']
file_means['io_time'] = file_means['read_io_time'] + file_means['write_io_time']


In [ ]:
# M2M overhead is the sum of the travel time of the reserve, sync_prov operations requests and acks
tcpstream_means['write_m2m_overhead'] = tcpstream_means['write_reserve_req_time'] + tcpstream_means['write_reserve_ack_time'] + tcpstream_means['write_sync_prov_req_time'] + tcpstream_means['write_sync_prov_ack_time']

tcpstream_means['m2m_overhead'] = tcpstream_means['write_m2m_overhead']

# Remote middleware overhead is the sum of the processing time of the reserve and sync_prov operations
tcpstream_means['write_middleware_remote_overhead'] = tcpstream_means['write_reserve_processing_time'] + tcpstream_means['write_sync_prov_processing_time']


In [ ]:
# P2M overhead is the sum of the travel time of the io_request and io_report operations requests and acks
tcpstream_means['read_p2m_overhead'] = tcpstream_means['read_io_request_req_time'] + tcpstream_means['read_io_report_req_time'] + tcpstream_means['read_io_request_ack_time'] + tcpstream_means['read_io_report_ack_time']
tcpstream_means['write_p2m_overhead'] =  tcpstream_means['write_io_request_req_time'] + tcpstream_means['write_io_report_req_time'] + tcpstream_means['write_io_request_ack_time'] + tcpstream_means['write_io_report_ack_time']

tcpstream_means['p2m_overhead'] = tcpstream_means['read_p2m_overhead'] + tcpstream_means['write_p2m_overhead']

file_means['read_p2m_overhead'] = file_means['read_io_request_req_time'] + file_means['read_io_report_req_time'] + file_means['read_io_request_ack_time'] + file_means['read_io_report_ack_time']
file_means['write_p2m_overhead'] = file_means['write_io_request_req_time'] + file_means['write_io_report_req_time'] + file_means['write_io_request_ack_time'] + file_means['write_io_report_ack_time']

file_means['p2m_overhead'] = file_means['read_p2m_overhead'] + file_means['write_p2m_overhead']

In [ ]:
# Middleware overhead is the sum of the processing time of the io_request and io_report operations and the remote overhead
# M2M overhead is subtracted as it is already included in the io_request and io_report operations processing time measurements
tcpstream_means['read_middleware_overhead'] = tcpstream_means['read_io_request_processing_time'] + tcpstream_means['read_io_report_processing_time']
tcpstream_means['write_middleware_overhead'] = tcpstream_means['write_io_request_processing_time'] + tcpstream_means['write_io_report_processing_time'] + tcpstream_means['write_middleware_remote_overhead'] - tcpstream_means['write_m2m_overhead']
tcpstream_means['middleware_overhead'] = tcpstream_means['read_middleware_overhead'] + tcpstream_means['write_middleware_overhead']

file_means['read_middleware_overhead'] = file_means['read_io_request_processing_time'] + file_means['read_io_report_processing_time']
file_means['write_middleware_overhead'] = file_means['write_io_request_processing_time'] + file_means['write_io_report_processing_time']
file_means['middleware_overhead'] = file_means['read_middleware_overhead'] + file_means['write_middleware_overhead']


In [ ]:
labels = ['File', 'TcpStream']
x = np.arange(len(labels))
width = 0.35  # width of the bars

# Create figure and axis
fig, ax = plt.subplots(figsize=(10, 6))

# Std IO exec time
std_io_time_bars = np.array([file_means['io_time'], tcpstream_means['io_time']])
ax.bar(x - width/2, std_io_time_bars, width, label='Standard I/O Execution Time', color='lightsalmon')

# Stde2e IO exec time
stde2e_io_time_bars = np.array([
                        file_means['io_time'] + file_means['middleware_overhead'] + file_means['p2m_overhead'],
                        tcpstream_means['io_time'] + tcpstream_means['middleware_overhead'] + tcpstream_means['p2m_overhead'] + tcpstream_means['m2m_overhead']
                    ])
ax.bar(x + width/2, stde2e_io_time_bars, width, label='Trace2e I/O Execution Time (stde2e)', color='lightblue')

# Add labels, title and legend
ax.set_xlabel('Resource Type')
ax.set_ylabel('Time (microseconds)')
ax.set_title('Comparison of I/O Libraries Performance for Read + Write Execution: Standard vs TracE2E')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()

# Add values on top of the bars
for i, v in enumerate(std_io_time_bars):
    ax.text(i - width/2, v, f'{v:.0f}', ha='center', va='bottom')
for i, v in enumerate(stde2e_io_time_bars):
    ax.text(i + width/2, v, f'{v:.0f}', ha='center', va='bottom')

# Add a grid to make it easier to read values
ax.grid(axis='y', linestyle='--', alpha=0.7)


plt.tight_layout()
plt.show()

In [ ]:
labels = ['File', 'TcpStream']
x = np.arange(len(labels))
width = 0.35

# Create figure and axis
fig, ax = plt.subplots(figsize=(9, 6))

# Std IO exec time
std_io_time_bars = np.array([file_means['read_io_time'], tcpstream_means['read_io_time']])
ax.bar(x - width/2, std_io_time_bars, width, label='Standard Read Execution Time', color='lightsalmon')

# Stde2e IO exec time
stde2e_io_time_bars = np.array([
                        file_means['read_io_time'] + file_means['read_middleware_overhead'] + file_means['read_p2m_overhead'],
                        tcpstream_means['read_io_time'] + tcpstream_means['read_middleware_overhead'] + tcpstream_means['read_p2m_overhead']
                    ])
ax.bar(x + width/2, stde2e_io_time_bars, width, label='Trace2e Read Execution Time (stde2e)', color='lightblue')

# Add labels, title and legend
ax.set_xlabel('Resource Type')
ax.set_ylabel('Time (microseconds)')
ax.set_title('Comparison of I/O Libraries Read Performance: Standard vs TracE2E')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()

# Add values on top of the bars
for i, v in enumerate(std_io_time_bars):
    ax.text(i - width/2, v, f'{v:.0f}', ha='center', va='bottom')
for i, v in enumerate(stde2e_io_time_bars):
    ax.text(i + width/2, v, f'{v:.0f}', ha='center', va='bottom')


# Add a grid to make it easier to read values
ax.grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

In [ ]:
labels = ['File', 'TcpStream']
x = np.arange(len(labels))
width = 0.35

# Create figure and axis
fig, ax = plt.subplots(figsize=(9, 6))

# Std IO exec time
std_io_time_bars = np.array([file_means['write_io_time'], tcpstream_means['write_io_time']])
ax.bar(x - width/2, std_io_time_bars, width, label='Standard Write Execution Time', color='lightsalmon')

# Stde2e IO exec time
stde2e_io_time_bars = np.array([
                        file_means['write_io_time'] + file_means['write_middleware_overhead'] + file_means['write_p2m_overhead'],
                        tcpstream_means['write_io_time'] + tcpstream_means['write_middleware_overhead'] + tcpstream_means['write_p2m_overhead'] + tcpstream_means['write_m2m_overhead']
                    ])
ax.bar(x + width/2, stde2e_io_time_bars, width, label='Trace2e Write Execution Time (stde2e)', color='lightblue')

# Add labels, title and legend
ax.set_xlabel('Resource Type')
ax.set_ylabel('Time (microseconds)')
ax.set_title('Comparison of I/O Libraries Write Performance: Standard vs TracE2E')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()

# Add values on top of the bars
for i, v in enumerate(std_io_time_bars):
    ax.text(i - width/2, v, f'{v:.0f}', ha='center', va='bottom')

for i, v in enumerate(stde2e_io_time_bars):
    ax.text(i + width/2, v, f'{v:.0f}', ha='center', va='bottom')

# Add a grid to make it easier to read values
ax.grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

In [ ]:
labels = ['File', 'TcpStream']
x = np.arange(len(labels))
width = 0.2

# Create figure and axis
fig, ax = plt.subplots(figsize=(9, 6))

# Bar positions
pos1 = x - width*1.5
pos2 = x - width*0.5
pos3 = x + width*0.5
pos4 = x + width*1.5

# Standard Read IO execution time
std_read_io_time_bars = np.array([file_means['read_io_time'], tcpstream_means['read_io_time']])
ax.bar(pos1, std_read_io_time_bars, width, label='Standard Read Time', color='wheat')

# Trace2e Read IO execution time
stde2e_read_io_time_bars = np.array([
    file_means['read_io_time'] + file_means['read_middleware_overhead'] + file_means['read_p2m_overhead'],
    tcpstream_means['read_io_time'] + tcpstream_means['read_middleware_overhead'] + tcpstream_means['read_p2m_overhead']
])
ax.bar(pos2, stde2e_read_io_time_bars, width, label='Trace2e Read Time', color='lightblue')

# Standard Write IO execution time
std_write_io_time_bars = np.array([file_means['write_io_time'], tcpstream_means['write_io_time']])
ax.bar(pos3, std_write_io_time_bars, width, label='Standard Write Time', color='orange')

# Trace2e Write IO execution time
stde2e_write_io_time_bars = np.array([
    file_means['write_io_time'] + file_means['write_middleware_overhead'] + file_means['write_p2m_overhead'],
    tcpstream_means['write_io_time'] + tcpstream_means['write_middleware_overhead'] + tcpstream_means['write_p2m_overhead'] + tcpstream_means['write_m2m_overhead']
])
ax.bar(pos4, stde2e_write_io_time_bars, width, label='Trace2e Write Time', color='cornflowerblue')

# Add labels, title and legend
ax.set_xlabel('Resource Type', fontsize=12)
ax.set_ylabel('Time (microseconds)', fontsize=12)
ax.set_title('I/O Performance Comparison: Standard vs TracE2E (Read & Write)', fontsize=14)
ax.set_xticks(x)
ax.set_xticklabels(labels, fontsize=11)
ax.legend(fontsize=10)

# Add values on top of the bars
def add_bar_labels(positions, values):
    for i, v in enumerate(values):
        ax.text(positions[i], v, f'{v:.0f}', ha='center', va='bottom', fontsize=9)

add_bar_labels(pos1, std_read_io_time_bars)
add_bar_labels(pos2, stde2e_read_io_time_bars)
add_bar_labels(pos3, std_write_io_time_bars)
add_bar_labels(pos4, stde2e_write_io_time_bars)

# Add a grid to make it easier to read values
ax.grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

In [ ]:
labels = ['File', 'TcpStream']
x = np.arange(len(labels))
width = 0.2

# Create figure and axis
fig, ax = plt.subplots(figsize=(9, 6))

# Bar positions
pos1 = x - width*1.5
pos2 = x - width*0.5
pos3 = x + width*0.5
pos4 = x + width*1.5

# Standard Read IO execution time (normalized to 1)
std_read_io_time_actual = np.array([file_means['read_io_time'], tcpstream_means['read_io_time']])
std_read_io_time_bars = np.ones_like(std_read_io_time_actual)  # Normalized to 1
ax.bar(pos1, std_read_io_time_bars, width, label='Standard Read Time', color='wheat')

# Trace2e Read IO execution time (normalized)
stde2e_read_io_time_bars = np.array([
    (file_means['read_io_time'] + file_means['read_middleware_overhead'] + file_means['read_p2m_overhead']) / file_means['read_io_time'],
    (tcpstream_means['read_io_time'] + tcpstream_means['read_middleware_overhead'] + tcpstream_means['read_p2m_overhead']) / tcpstream_means['read_io_time']
])
ax.bar(pos2, stde2e_read_io_time_bars, width, label='Trace2e Read Time', color='lightblue')

# Standard Write IO execution time (normalized to 1)
std_write_io_time_actual = np.array([file_means['write_io_time'], tcpstream_means['write_io_time']])
std_write_io_time_bars = np.ones_like(std_write_io_time_actual)  # Normalized to 1
ax.bar(pos3, std_write_io_time_bars, width, label='Standard Write Time', color='orange')

# Trace2e Write IO execution time (normalized)
stde2e_write_io_time_bars = np.array([
    (file_means['write_io_time'] + file_means['write_middleware_overhead'] + file_means['write_p2m_overhead']) / file_means['write_io_time'],
    (tcpstream_means['write_io_time'] + tcpstream_means['write_middleware_overhead'] + tcpstream_means['write_p2m_overhead'] + tcpstream_means['write_m2m_overhead']) / tcpstream_means['write_io_time']
])
ax.bar(pos4, stde2e_write_io_time_bars, width, label='Trace2e Write Time', color='cornflowerblue')

# Add labels, title and legend
ax.set_xlabel('Resource Type', fontsize=12)
ax.set_ylabel('Normalized Time (Standard I/O = 1.0)', fontsize=12)
ax.set_title('I/O Performance Comparison: Standard vs TracE2E (Normalized)', fontsize=14)
ax.set_xticks(x)
ax.set_xticklabels(labels, fontsize=11)
ax.legend(fontsize=10)

# Add values on top of the bars
def add_bar_labels(positions, values):
    for i, v in enumerate(values):
        ax.text(positions[i], v, f'x {v:.2f}', ha='center', va='bottom', fontsize=9)

add_bar_labels(pos2, stde2e_read_io_time_bars)
add_bar_labels(pos4, stde2e_write_io_time_bars)

# Add a horizontal line at y=1 to emphasize the baseline
ax.axhline(y=1, color='gray', linestyle='-', alpha=0.3)

# Add a grid to make it easier to read values
ax.grid(axis='y', linestyle='--', alpha=0.7)

# Set y-axis to start from 0
ax.set_ylim(bottom=0)

plt.tight_layout()
plt.show()

In [ ]:
labels = ['File', 'TcpStream']
x = np.arange(len(labels))
width = 0.35  # width of the bars

# Create figure and axis
fig, ax = plt.subplots(figsize=(10, 6))

# Stde2e IO exec time
stde2e_io_time_bars = np.array([
                        file_means['io_time'] + file_means['middleware_overhead'] + file_means['p2m_overhead'],
                        tcpstream_means['io_time'] + tcpstream_means['middleware_overhead'] + tcpstream_means['p2m_overhead'] + tcpstream_means['m2m_overhead']
                    ])
ax.bar(x - width*0.75, stde2e_io_time_bars, width*0.5, label='Total Trace2e I/O Execution Time', color='lightblue')
# For the stacked bars
# First layer: io_time
std_io_time_bars = np.array([file_means['io_time'], tcpstream_means['io_time']])
ax.bar(x + width*0.25, 
        std_io_time_bars, 
        width*1.5,
        label='including Call to Standard Library', 
        color='lightsalmon')

# Second layer: middleware_overhead
middleware_overhead_bars = np.array([file_means['middleware_overhead'], tcpstream_means['middleware_overhead']])
ax.bar(x + width*0.25, 
        middleware_overhead_bars, 
        width*1.5, 
        bottom=[file_means['io_time'], tcpstream_means['io_time']], 
        label='including Middleware Overhead', 
        color='mediumaquamarine')

# Third layer: p2m_overhead
p2m_overhead_bars = np.array([file_means['p2m_overhead'], tcpstream_means['p2m_overhead']])
ax.bar(x + width*0.25, 
    p2m_overhead_bars, 
    width*1.5, 
    bottom=std_io_time_bars+middleware_overhead_bars, 
    label='including P2M Communication Overhead', 
    color='gold')


# Fourth layer: m2m_overhead
m2m_overhead_bars = np.array([0, tcpstream_means['write_m2m_overhead']])
ax.bar(x + width*0.25, 
    m2m_overhead_bars, 
    width*1.5, 
    bottom=std_io_time_bars+middleware_overhead_bars+p2m_overhead_bars, 
    label='including M2M Communication Overhead', 
    color='lightgreen')

ax.set_xlabel('Resource Type')
ax.set_ylabel('Time (microseconds)')
ax.set_title('TracE2E I/O Overhead Analysis')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()

# Add values in the middle of the bars
for i, v in enumerate(stde2e_io_time_bars):
    ax.text(i - width*0.75, v*0.5, f'{v:.0f}', ha='center', va='bottom')

for i, v in enumerate(std_io_time_bars):
    ax.text(i + width*0.25, v*0.5, f'{v:.0f}', ha='center', va='center', color='black')

for i, v in enumerate(middleware_overhead_bars):
    ax.text(i + width*0.25, std_io_time_bars[i] + v*0.5, f'{v:.0f}', ha='center', va='center', color='black')

for i, v in enumerate(p2m_overhead_bars):
    ax.text(i + width*0.25, std_io_time_bars[i] + middleware_overhead_bars[i] + v*0.5, f'{v:.0f}', ha='center', va='center', color='black')

for i, v in enumerate(m2m_overhead_bars):
    if v > 0:
        ax.text(i + width*0.25, std_io_time_bars[i] + middleware_overhead_bars[i] + p2m_overhead_bars[i] + v*0.5, f'{v:.0f}', ha='center', va='center', color='black')


# Add a grid to make it easier to read values
ax.grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()